Homework 1

In [ ]:
from search import Problem, hill_climbing, simulated_annealing, exp_schedule
from csp import backtracking_search, mrv, forward_checking, parse_neighbors, CSP, min_conflicts
import random

## Question 1: Introspection in AI?

Introspection as I understand it has largely been abandoned by psychology in any attempt to further our understanding of our own minds. It appears that we are not actually well enough aware of our own mental processes to explain them in a way that science can take advantage of. If it doesn't seem useful to psychology, I have a feeling it would prove quite challenging to attempt to describe our mental processes in a way that an AI could make better sense of it. One challenge would simply be the transcription. We could write down our thoughts, and perhaps an AI could do something with them or generate similar-ish thoughts, but the input/output seems to be limited to text-based interfaces which seem quite limiting for any useful applications. However even if we could get a computer to generate similar such "thoughts" I suspect that the actual generation of the thoughts in our brains is beneath the thought-level that we typically have access to and is reliant on subconscious processes that would be missing in an AI built off of surface level thoughts.

## Question 2: Travelling Salesman Problem

I represented my state simply as a list of cities and my actions then were random swappings of cities. So each action set being generated was selecting a random city and swapping it with n (generally 10) potential candidate cities. (technically this was up to n since it could randomly choose a city being swapped in a previous action). And the cost or value of each state was just the combined length of the journey.

In [3]:
class TSP(Problem):

    # constructor
    def __init__(self, initial, distances, n=10):
        self.initial = initial
        self.distances = distances
        self.n = n

    # some random actions from the given state
    def actions(self, state):
        actions = []
        if state is not None:

            # choose up to n-1 random positions to swap with position a
            a = random.randrange(1, len(state)-1)
            for i in range(1, self.n-1):
                b = random.randrange(1, len(state)-1)
                while a == b:
                    b = random.randrange(1, len(state)-1)
                actions.append([a, b])

            # add an additional swap which will swap the start/ending points with some other option
            actions.append(['swap', random.randrange(1, len(state)-1)])

        return actions

    # the resulting state
    def result(self, state, action):

        new_state = state[:]

        # swapping the endpoints out for a middle point
        if action[0] == 'swap':
            temp = new_state[action[1]]
            new_state[action[1]] = new_state[0]
            new_state[len(state)-1] = temp
            new_state[0] = temp
        # just swapping the two given points
        else:
            temp = new_state[action[0]]
            new_state[action[0]] = new_state[action[1]]
            new_state[action[1]] = temp

        return new_state

    # the distance of the trip
    def value(self, cities):
        cost = 0
        last_city = None
        for city in cities:
            if last_city is None:
                last_city = city
            else:
                cost += self.distances[str(last_city) + " --> " + str(city)]
                last_city = city

        return -cost

To start off with I defined a small world problem with 4 "cities" as the letters A-D and the distances 1-6 between each. The best solutions after going through the permutations by hand were ABDC and CDBA.

In [4]:
    cities = ['D', 'A', 'C', 'B', 'D']
    distances_between_cities = {
        "A --> B": 1,
        "B --> A": 1,
        "A --> D": 5,
        "D --> A": 5,
        "A --> C": 4,
        "C --> A": 4,
        "B --> C": 6,
        "C --> B": 6,
        "B --> D": 2,
        "D --> B": 2,
        "C --> D": 3,
        "D --> C": 3
    }

So I selected one of the worst states for the initial state

In [5]:
tsp = TSP(cities, distances_between_cities)
print("initial: " + str(cities) + "\tvalue: " +str(tsp.value(cities)))


initial: ['D', 'A', 'C', 'B', 'D']	value: -17


Thankfully, however, Hill-climbing and Simulated-Annealing can find much better options than the default.

In [6]:
def compare_algorithms_for_this_world(world, dist_dict, attempts=30, print_solution=False):

    # using the initial as our baseline
    best_hill_climber = world
    best_annealing = world

    # initializing averages
    avg_hill_climber = 0
    avg_annealing = 0

    # our small world problem representation
    tsp = TSP(world, dist_dict)

    # try both algorithms over and over
    for i in range(0, attempts):

        hill_climber = hill_climbing(tsp)
        annealing = simulated_annealing(tsp, exp_schedule(k=20, lam=0.005, limit=1000))

        # keep track of the best solutions
        if tsp.value(hill_climber) > tsp.value(best_hill_climber):
                best_hill_climber = hill_climber
        if tsp.value(annealing) > tsp.value(best_annealing):
            best_annealing = annealing

        # maintain average values
        avg_hill_climber += tsp.value(hill_climber)
        avg_annealing += tsp.value(annealing)

    # Calculate the average
    avg_hill_climber /= attempts
    avg_annealing /= attempts

    # Ouptut: world statistics
    print("World of size " + str(len(world)) + " with " + str(attempts) + " per algorithm")

    hill_path = str(best_hill_climber)
    annealing_path = str(best_annealing)

    if not print_solution:
        hill_path = ""
        annealing_path = ""

    # Output: hill climbing results
    print("Hill Climbing:\t" + hill_path +
          "\tbest: " + str(tsp.value(best_hill_climber)) +
          "\tavg: " + str(round(avg_hill_climber, 2)))

    # Output: simulated annealing results
    print("Annealing:\t" + annealing_path +
          "\tbest: " + str(tsp.value(best_annealing)) +
          "\tavg: " + str(round(avg_annealing, 2)))

# test out the small world
compare_algorithms_for_this_world(cities, distances_between_cities, print_solution=True)

World of size 5 with 30 per algorithm
Hill Climbing:	['D', 'C', 'A', 'B', 'D']	best: -10	avg: -10.0
Annealing:	['B', 'D', 'C', 'A', 'B']	best: -10	avg: -10.0


So while after 30 tries they've both almost always found one of the best solutions, it does appear that hill climbing appears to miss the mark a little more often, resulting in an average further from the optimal solution. Granted, for such a small world it's not surprising that they both are able to find an optimal solution pretty quickly.

However, to create slightly larger and more complicated worlds, we can generate completely random scenarios for us to play around with as well. Beneath is a fictitious world generator.

In [7]:
def generate_world(size=10, max_distance=10):

    # generating the world
    world = []
    for i in range(0, size):
        world.append(i)

    # generating the distance map
    distances = {}
    for city in world:
        other_cities = world[:]
        for other_city in other_cities:
            dist = random.randrange(1, max_distance)
            distances[str(city) + ' --> ' + str(other_city)] = dist
            distances[str(other_city) + ' --> ' + str(city)] = dist

    # return the generated world and distances dictionary
    return world, distances

# generate and try out some larger worlds
world_sizes = [4, 8, 16, 32, 64, 128]
for world_size in world_sizes:
    cities, distances = generate_world(world_size)
    compare_algorithms_for_this_world(cities, distances)


World of size 4 with 30 per algorithm
Hill Climbing:		best: -14	avg: -14.0
Annealing:		best: -14	avg: -14.0
World of size 8 with 30 per algorithm
Hill Climbing:		best: -15	avg: -18.93
Annealing:		best: -16	avg: -16.0
World of size 16 with 30 per algorithm
Hill Climbing:		best: -43	avg: -57.47
Annealing:		best: -34	avg: -38.1
World of size 32 with 30 per algorithm
Hill Climbing:		best: -91	avg: -118.17
Annealing:		best: -48	avg: -61.27
World of size 64 with 30 per algorithm
Hill Climbing:		best: -215	avg: -265.2
Annealing:		best: -132	avg: -141.97
World of size 128 with 30 per algorithm
Hill Climbing:		best: -433	avg: -543.1
Annealing:		best: -265	avg: -302.23


So especially for the larger world-sizes, the simulated annealing technique seems to do significantly better, though on occasion I have seen Hill-climbing doing a smidgen better with the smaller worlds. This is probably due to the potential for simulated annealing to take potentially unoptimal paths in the hopes of a better path down the road.

In [8]:
## Question 3: Scheduling

In [9]:
# options for the classes
classes = 'CS-108 CS-112 CS-212 CS-214 CS-344 CS-384 CS-398'.split()
rooms = 'NH253 SB382'.split()
time_slots = 'MWF800 MWF900 MWF1030 TTh1030 TTh12:05'.split()
professors = 'Adams Plantinga Schuurman Vanderlinden'.split()

So the variables for this problem are the classes (i.e. CS-108) and the domain is the set of combinations of professors-rooms-times.

In [10]:
# all the domain possibilities
domain_possibilities = []
for room in rooms:
    for time_slot in time_slots:
        for prof in professors:
            domain_possibilities.append(prof + " " + room + " " + time_slot)

# variables : courses
# domains : everything else
variables = classes
domains = {}
for var in variables:
    domains[var] = domain_possibilities


While we can't use neighbors to trim down the size of the problem-space, we still apparently need to supply the neighbors such that everything touches everything so that the CSP algorithms actually compare everyone. You will also note that instead of trying to figure out what an empty neighbors data structure looks like, I just hard-coded a dummy neighbor before linking everybody else up.

In [11]:
# neighbors : everybody is a neighbor
neighbors = parse_neighbors("""CS-344: NH158 8MWF""", variables)
for type in [classes, domain_possibilities]:
    for A in type:
        for B in type:
            if A != B:
                if B not in neighbors[A]:
                    neighbors[A].append(B)
                if A not in neighbors[B]:
                    neighbors[B].append(A)

The constraint function handles both the requirements that no professor is in two rooms at once, or that two classes meet in the same room at once. It also limits which courses can be taught by whom. It would have been possible to have pre-limited the domain so that the course assignments were respected prior to the domains creation, but this seemed simpler. Especially in larger problems, being able to shrink the domain would probably be a better solution when it is possible simply so that fewer things need to be considered in the already large problem-space.

In [12]:
# constraint function
# ensuring that a room is acceptable
# and that no schedule conflicts occur 
# (i.e. kvlinden can't be in two rooms at once)
def scheduler_constraint(A, a, B, b, recurse=0):
    prof_a = a.split()[0]
    room_a = a.split()[1]
    time_a = a.split()[2]

    prof_b = b.split()[0]
    room_b = b.split()[1]
    time_b = b.split()[2]

    # Schuurman must teach 108
    if (A == "CS-108" and prof_a != "Schuurman") or (B == "CS-108" and prof_b != "Schuurman"):
        return False

    # Adams must teach 112
    if (A == "CS-112" and prof_a != "Adams") or (B == "CS-112" and prof_b != "Adams"):
        return False

    # Plantinga must teach 212
    if (A == "CS-212" and prof_a != "Plantinga") or (B == "CS-212" and prof_b != "Plantinga"):
        return False

    # Adams must teach 214
    if (A == "CS-214" and prof_a != "Adams") or (B == "CS-214" and prof_b != "Adams"):
        return False

    # Vanderlinden must teach 344
    if (A == "CS-344" and prof_a != "Vanderlinden") or (B == "CS-344" and prof_b != "Vanderlinden"):
        return False

    # Schuurman must teach 384
    if (A == "CS-384" and prof_a != "Schuurman") or (B == "CS-384" and prof_b != "Schuurman"):
        return False

    # Vanderlinden must teach 398
    if (A == "CS-398" and prof_a != "Vanderlinden") or (B == "CS-398" and prof_b != "Vanderlinden"):
        return False

    # professor can only teach one course at a time
    if(prof_a == prof_b) and (time_a == time_b):
        return False

    # we can't have two courses in the same room at the same time
    if(room_a == room_b) and (time_a == time_b):
        return False

    return True

Finally, we can actually run the problem. I should note that the strategy I approached this with, the anti-Zebra method, was mostly gleaned from hints you gave in class and discussions I had with Brent and Logan. Approaching the problem this way feels a little more natural where the variables are more clearly what I would consider variables, rather than the property-tagging nature of the Zebra approach.

In [16]:
problem = CSP(variables, domains, neighbors, scheduler_constraint)
solution = backtracking_search(problem)
print(solution)
solution = min_conflicts(problem)
print(solution)

{'CS-108': 'Schuurman NH253 MWF800', 'CS-344': 'Vanderlinden NH253 TTh12:05', 'CS-112': 'Adams NH253 MWF900', 'CS-212': 'Plantinga NH253 MWF1030', 'CS-384': 'Schuurman SB382 MWF900', 'CS-398': 'Vanderlinden SB382 MWF800', 'CS-214': 'Adams NH253 TTh1030'}
{'CS-108': 'Schuurman NH253 TTh1030', 'CS-344': 'Vanderlinden NH253 MWF1030', 'CS-112': 'Adams SB382 MWF1030', 'CS-212': 'Plantinga SB382 MWF800', 'CS-384': 'Schuurman NH253 MWF800', 'CS-398': 'Vanderlinden NH253 MWF900', 'CS-214': 'Adams NH253 TTh12:05'}
